# How to deal with timestamps

## Install

In [ ]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
!pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="pandas-test-domain-1", port="auto", reset=True)

# Data owner: upload dataset

In [ ]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

## Load data

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at /var/log/popularity-contest.

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from syft.service.project.project import Project
from syft.util.util import autocache, PANDAS_DATA

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [ ]:
popcon = pd.read_csv(autocache(f"{PANDAS_DATA}/popularity-contest"), sep=' ', )[:-1]
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

In [ ]:
set(popcon["tag"])

## Create mock data

Lets create a mock dataset

In [ ]:
# weather_types = set(weather_2012_final.Weather.to_list())

In [ ]:
from random import randint
import random

In [ ]:
def get_random_timestamp():
    return "135" + "".join([str(randint(0,9)) for i in range(6)])

In [ ]:
def get_mock_date(i):
    return str(parse('Jun 1 2010') + timedelta(days=i))

def get_mock_row(i):
    res = dict()
    for k, function in mock_functions.items():
        res[k] = function()
    return res
    

In [ ]:
mock_functions = {'atime': lambda: get_random_timestamp(),
                  'ctime': lambda: get_random_timestamp(),
                  'package-name': lambda: random.choice([
                      'libghc-stm-dev',
                      'libqtdee-dev',
                      'geoclue-ubuntu-geoip',
                      'libdesktop-agnostic0',
                      'ubuntu-extras-keyring',
                      'libbsd0',
                      'libxres-dev']),
                  'mru-program': lambda: random.choice([
                      '/usr/bin/opam',
                      '/usr/bin/onboard',
                      '/lib/init/upstart-job',
                      '/usr/bin/page',
                  ]),
                  'tag': lambda: random.choice(['<OLD>', '<RECENT-CTIME>', 'nan'])}


In [ ]:
mock = pd.DataFrame(data=[get_mock_row(i) for i in range(len(popcon))],
                    columns=popcon.columns)
    


Upload the data

In [ ]:
dataset = sy.Dataset(name="test", asset_list=[sy.Asset(name=f"weather", data=popcon,
                                                       mock=mock, mock_is_real=False)])
root_domain_client.upload_dataset(dataset)


In [ ]:
popcon.head()

## Create user account

In [ ]:
user = root_domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")
# todo: give user data scientist role
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: create syft_function

In [ ]:
import numpy as np
import pandas as pd

## Summary

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

## Get mocks

In [ ]:
ds = guest_domain_client.datasets[0]

In [ ]:
asset = ds.assets[0]

In [ ]:
popcon = asset.mock.syft_action_data

## Parsing Unix timestamps

The colums are the access time, created time, package name, recently used program, and a tag



In [ ]:
popcon[:5]

The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [ ]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [ ]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [ ]:
popcon['atime'].dtype

So now we can look at our `atime` and `ctime` as dates!

In [ ]:
popcon[:5]

Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.


In [ ]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [ ]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [ ]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

## Putting it together

Now we want to request the full code execution.

Let's put all that together, to prove how easy it is. 6 lines of magical pandas!

In [ ]:
@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]

Request code execution

In [ ]:
req = guest_domain_client.api.services.code.request_code_execution(find_recently_installed)

In [ ]:
submitted_code = guest_domain_client.code[0]

In [ ]:
assert guest_domain_client.api.services.code.get_all()

Create and submit project

In [ ]:
new_project = sy.Project(name="Pandas Chapter 8",
                         description="Hi, I would like to get some insights about the installed programs")

In [ ]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [ ]:
guest_domain_client.submit_project(new_project)

# Data owner: execute syft_function

In [ ]:
from syft import MessageStatus

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

# Get messages

In [ ]:
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)

In [ ]:
messages

In [ ]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [ ]:
request = project_message.link.requests[0]
func = request.changes[0].link
op = func.output_policy_type

In [ ]:
func

In [ ]:
find_recently_installed = func.unsafe_function

In [ ]:
real_data = domain_client.datasets[0].assets[0].data.syft_action_data

In [ ]:
real_data

In [ ]:
real_result = find_recently_installed(popcon=real_data)

In [ ]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)

# Data Owner: fetch result

In [ ]:
guest_client.api.services.code[0].status

In [ ]:
asset = guest_client.datasets[0].assets[0]

In [ ]:
real_result = guest_client.api.services.code.find_recently_installed(popcon=asset)

In [ ]:
real_result

In [ ]:
node.land()